In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
import PIL.Image
import os
import io
import matplotlib.pyplot as plt # Import library untuk grafik baru

# --- KONFIGURASI PATH DATASET ---
ROOT_DATASET_PATH = "/content/drive/MyDrive/FP_AI/images"

# 1. KONFIGURASI HALAMAN & CSS
st.set_page_config(page_title="WikiArt Generator Pro", layout="wide", page_icon="🎨")

st.markdown("""
<style>
    @import url('https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;600;700&display=swap');
    html, body, [class*="css"] { font-family: 'Poppins', sans-serif; }
    .stApp { background-color: #0E1117; color: #FAFAFA; }
    .main-header {
        text-align: center; font-weight: 700; font-size: 3rem;
        background: linear-gradient(to right, #C33764 0%, #1D2671 100%);
        -webkit-background-clip: text; -webkit-text-fill-color: transparent; margin-bottom: 10px;
    }
    .stButton > button {
        width: 100%; background: linear-gradient(90deg, #C33764 0%, #1D2671 100%);
        border: none; color: white; padding: 15px 32px;
        font-size: 16px; font-weight: 600; border-radius: 12px; transition: 0.3s;
    }
    .stButton > button:hover { transform: translateY(-3px); box-shadow: 0px 10px 20px rgba(195, 55, 100, 0.4); }
</style>
""", unsafe_allow_html=True)

st.markdown('<h1 class="main-header">🏛️ The Art Museum Generator</h1>', unsafe_allow_html=True)
st.write("<p style='text-align: center; color: #aaa;'>Akses ribuan koleksi seni dunia dan terapkan gayanya pada fotomu.</p>", unsafe_allow_html=True)

# 2. LOGIKA DATASET
st.sidebar.title("🗂️ Kurasi Museum")

if not os.path.exists(ROOT_DATASET_PATH):
    st.error(f"❌ Error: Path tidak ditemukan! \n\nCek variabel ROOT_DATASET_PATH di kode. \nPath saat ini: {ROOT_DATASET_PATH}")
    st.stop()

try:
    artists = sorted([d for d in os.listdir(ROOT_DATASET_PATH) if os.path.isdir(os.path.join(ROOT_DATASET_PATH, d))])
    selected_artist = st.sidebar.selectbox("1. Pilih Seniman:", artists)
except Exception as e:
    st.sidebar.error(f"Gagal membaca folder dataset: {e}")
    selected_artist = None

selected_style_path = None

if selected_artist:
    artist_folder = os.path.join(ROOT_DATASET_PATH, selected_artist)
    try:
        artworks = sorted([f for f in os.listdir(artist_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        if artworks:
            selected_artwork = st.sidebar.selectbox("2. Pilih Karya:", artworks)
            selected_style_path = os.path.join(artist_folder, selected_artwork)
            st.sidebar.image(selected_style_path, caption=f"🎨 {selected_artwork}", use_container_width=True)
        else:
            st.sidebar.warning("Folder seniman ini kosong.")
    except Exception as e:
        st.sidebar.error(f"Error membaca file: {e}")

# 3. CORE DEEP LEARNING
def load_img(image_file):
    max_dim = 512
    if isinstance(image_file, str):
        img = tf.io.read_file(image_file)
        img = tf.image.decode_image(img, channels=3)
    else:
        img = PIL.Image.open(image_file)
        img = np.array(img)

    img = tf.convert_to_tensor(img)
    img = tf.image.convert_image_dtype(img, tf.float32)
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim
    new_shape = tf.cast(shape * scale, tf.int32)
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

@st.cache_resource
def get_model():
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    content_layers = ['block5_conv2']
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
    outputs = [vgg.get_layer(name).output for name in (style_layers + content_layers)]
    model = tf.keras.Model([vgg.input], outputs)
    return model, style_layers, content_layers, vgg

model, style_layers, content_layers, base_vgg = get_model()

with st.sidebar.expander("🔍 Lihat Arsitektur VGG19"):
    stream = io.StringIO()
    base_vgg.summary(print_fn=lambda x: stream.write(x + '\n'))
    summary_string = stream.getvalue()
    st.code(summary_string, language='text')

# 4. MAIN INTERFACE
uploaded_file = st.file_uploader("Upload Foto Kamu", type=['jpg', 'png', 'jpeg'])

if st.button("🚀 GENERATE ARTWORK") and uploaded_file and selected_style_path:
    col1, col2 = st.columns(2)
    with col1: st.image(uploaded_file, caption="Original Photo", use_container_width=True)

    progress_bar = st.progress(0)
    status_text = st.empty()

    with st.spinner('Sedang melukis... Tunggu Sebentar Yaaa:)'):
        content_image = load_img(uploaded_file)
        style_image = load_img(selected_style_path)

        def gram_matrix(input_tensor):
            result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
            input_shape = tf.shape(input_tensor)
            num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
            return result/(num_locations)

        class StyleContentModel(tf.keras.models.Model):
            def __init__(self, style_layers, content_layers):
                super(StyleContentModel, self).__init__()
                self.vgg = model
                self.style_layers = style_layers
                self.content_layers = content_layers
                self.num_style_layers = len(style_layers)
                self.vgg.trainable = False

            def call(self, inputs):
                inputs = inputs*255.0
                preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
                outputs = self.vgg(preprocessed_input)
                style_outputs, content_outputs = (outputs[:self.num_style_layers], outputs[self.num_style_layers:])
                style_outputs = [gram_matrix(style_output) for style_output in style_outputs]
                content_dict = {content_name: value for content_name, value in zip(self.content_layers, content_outputs)}
                style_dict = {style_name: value for style_name, value in zip(self.style_layers, style_outputs)}
                return {'content': content_dict, 'style': style_dict}

        extractor = StyleContentModel(style_layers, content_layers)
        style_targets = extractor(style_image)['style']
        content_targets = extractor(content_image)['content']
        image = tf.Variable(content_image)

        opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
        style_weight=1e-2
        content_weight=1e4

        @tf.function()
        def train_step(image):
            with tf.GradientTape() as tape:
                outputs = extractor(image)
                loss = tf.add_n([tf.reduce_mean((outputs['content'][name]-content_targets[name])**2) for name in outputs['content'].keys()])*content_weight / len(content_layers)
                loss += tf.add_n([tf.reduce_mean((outputs['style'][name]-style_targets[name])**2) for name in outputs['style'].keys()])*style_weight / len(style_layers)
                total_loss = loss + tf.reduce_sum(tf.image.total_variation(image))*30
            grad = tape.gradient(total_loss, image)
            opt.apply_gradients([(grad, image)])
            image.assign(tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0))
            return total_loss

        import time
        epochs = 5
        steps_per_epoch = 50
        total_steps = epochs * steps_per_epoch
        loss_history = []

        start_time = time.time()

        for n in range(epochs):
            for m in range(steps_per_epoch):
                loss_value = train_step(image)
                current_loss = float(loss_value)
                loss_history.append(current_loss)

                current_step = n * steps_per_epoch + m + 1
                progress = current_step / total_steps
                progress_bar.progress(progress)

                if m % 10 == 0:
                    status_text.write(f"⏳ **Epoch {n+1}/{epochs}** | Step {m}/{steps_per_epoch} | Loss: `{current_loss:.2f}`")

        status_text.success(f"✅ Selesai dalam {time.time() - start_time:.1f} detik!")
        progress_bar.empty()

        result = tensor_to_image(image)
        with col2:
            st.image(result, caption="AI Masterpiece", use_container_width=True)
            buf = io.BytesIO()
            result.save(buf, format="JPEG")
            st.download_button("📥 Download Result", data=buf.getvalue(), file_name="art_result.jpg", mime="image/jpeg")

        # GRAFIK MATPLOTLIB (MERAH & DOTS)
        st.write("---")
        # Membuat Figure Matplotlib
        fig, ax = plt.subplots(figsize=(10, 5))

        # Plot Data: Warna Merah ('red'), Marker Bulat ('o'), Ukuran Marker (4)
        ax.plot(loss_history, color='red', marker='o', linestyle='-', linewidth=2, markersize=4, label='Total Loss')

        # Dekorasi Grafik
        ax.set_title("Grafik Penurunan Loss ", fontsize=14, fontweight='bold')
        ax.set_xlabel("Steps ", fontsize=12)
        ax.set_ylabel("Nilai Loss", fontsize=12)
        ax.legend()
        ax.grid(True, linestyle='--', alpha=0.7) # Grid putus-putus

        # Tampilkan di Streamlit
        st.pyplot(fig)

elif not uploaded_file:
    st.info("👆 Upload foto dulu ya.")
elif not selected_style_path:
    st.warning("👈 Pilih seniman dan lukisan dulu di Sidebar.")

Overwriting app.py


In [ ]:
# 1. Install Streamlit & Cloudflared
!pip install streamlit -q
!wget -q -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

# 2. Jalankan Streamlit di BACKGROUND & Simpan Log
# Kita simpan log ke 'streamlit.log' supaya bisa dicek jika error
print("⏳ Sedang menjalankan Streamlit...")
!nohup streamlit run app.py --server.port 8501 > streamlit.log 2>&1 &

# Beri waktu 5 detik agar Streamlit loading dulu
import time
time.sleep(5)

# 3. Jalankan Cloudflare Tunnel dengan IP PASTI (127.0.0.1)
# Kita simpan log ke 'tunnel.log' agar linknya bisa diambil nanti
print("🔗 Sedang membuat Tunnel Cloudflare...")
!nohup ./cloudflared-linux-amd64 tunnel --url http://127.0.0.1:8501 > tunnel.log 2>&1 &

# 4. Tampilkan Link Akses
time.sleep(5)
print("\n✅ SELESAI! Silakan buka link di bawah ini:")
# Perintah ini akan mencari baris yang berisi link .trycloudflare.com di dalam log
!grep -o 'https://.*\.trycloudflare.com' tunnel.log

⏳ Sedang menjalankan Streamlit...
🔗 Sedang membuat Tunnel Cloudflare...

✅ SELESAI! Silakan buka link di bawah ini:
https://feeding-asked-tons-carry.trycloudflare.com
